In [1]:
import pandas as pd
import ast  # For safely evaluating string representations of lists

# Load quotes and users data
users_df = pd.read_csv('users.csv')
users_df = pd.read_csv('users_tags.csv')
# Safely convert 'likes' column (string representation of list) to actual lists
users_df['likes'] = users_df['likes'].apply(ast.literal_eval)

users_df.head() # Verify the likes column is properly converted


,name,likes
0,User1,"[1207, 445, 140, 2259, 1611, 2085, 973, 1920, ..."
1,User2,"[1, 154, 1022, 1925, 1263, 417, 228, 855, 664,..."
2,User3,"[968, 2614, 2573, 212, 2966, 1666, 638, 2206, ..."
3,User4,"[1996, 1306, 1231, 499, 1002, 2251, 1730, 124,..."
4,User5,"[1523, 2136, 442, 1733, 2880, 1756, 1442, 151,..."


In [2]:
transactions = users_df['likes'].tolist()

In [ ]:
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

# Transform transactions into one-hot encoded format
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_onehot = pd.DataFrame(te_ary, columns=te.columns_)

# Apply Apriori to find frequent itemsets
frequent_itemsets = apriori(df_onehot, min_support=0.01, use_colnames=True)

# Generate association rules
num_freq_itemsets = frequent_itemsets.shape[0]
rules = association_rules(frequent_itemsets,num_freq_itemsets, metric="confidence", min_threshold=0.5, support_only=False)

# Display rules
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])



In [ ]:
def recommend_quotes(user_likes, rules):
    # Filter rules based on user's liked quotes
    recommendations = []
    for _, rule in rules.iterrows():
        if set(rule['antecedents']).issubset(user_likes):
            recommendations.extend(rule['consequents'])

    # Remove quotes the user already likes
    recommendations = set(recommendations) - set(user_likes)
    return list(recommendations)

# Example: Recommend quotes for User1
user_likes = users_df.loc[users_df['name'] == 'User12', 'likes'].values[0]
recommendations = recommend_quotes(user_likes, rules)
print("Recommended quotes for User1:", recommendations)


Recommended quotes for User1: [641, 615, 684, 209, 601]
